In [ ]:
# analyze_missing_idescat_parallel_json.py

import requests
import json
import time
import pandas as pd
from collections import defaultdict
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# --- Configuration ---
INDICATORS_TO_CHECK = [ # Same list as before
    'f171', 'f36', 'f42', 'f182', 'f183', 'f191', 'f210', 'f240', 'f241',
    'f248', 'f271', 'f272', 'f328', 'f329', 'f3', 'f186', 'f187', 'f260',
    'f261', 'f262', 'f263', 'f264', 'f265', 'f266', 'f267', 'f268', 'f269',
    'f270', 'f273', 'f274', 'f275', 'f276', 'f277', 'f278', 'f279', 'f280',
    'f281', 'f282', 'f283', 'f284', 'f285', 'f286', 'f287', 'f288', 'f289',
    'f290', 'f291', 'f292', 'f293', 'f294', 'f295', 'f296', 'f297', 'f298',
    'f299', 'f300', 'f301', 'f302', 'f303', 'f304', 'f305', 'f306', 'f307',
    'f308', 'f309', 'f310', 'f311', 'f312', 'f313', 'f314', 'f315', 'f316',
    'f317', 'f318', 'f319', 'f320', 'f321', 'f322', 'f323', 'f324', 'f325',
    'f326', 'f327'
]
INDICATORS_QUERY_STRING = ','.join(INDICATORS_TO_CHECK)

API_BASE_URL = "https://api.idescat.cat/emex/v1"
MAX_WORKERS = 10 # Using the safer limit

# --- Helper Functions (Keep as before) ---

def get_all_municipality_ids() -> list:
    # (Same function as previously provided)
    print("Fetching list of all municipalities...")
    nodes_url = f"{API_BASE_URL}/nodes.json?tipus=mun"
    try:
        response = requests.get(nodes_url)
        response.raise_for_status()
        data = response.json()
        municipality_ids = []
        def extract_municipalities(node):
            ids = []
            if node.get('scheme') == 'mun':
                ids.append(node.get('id'))
            children = node.get('v', [])
            if isinstance(children, dict): children = [children]
            if isinstance(children, list):
                for child_node in children:
                    ids.extend(extract_municipalities(child_node))
            return ids
        root_nodes = data.get('fitxes', {}).get('v', [])
        if isinstance(root_nodes, dict): root_nodes = [root_nodes]
        for root_node in root_nodes:
             municipality_ids.extend(extract_municipalities(root_node))
        unique_ids = sorted(list(set(municipality_ids)))
        print(f"Found {len(unique_ids)} unique municipality IDs.")
        return unique_ids
    except Exception as e:
        print(f"Error fetching municipality list: {e}")
        return []

def fetch_indicators_for_municipality(municipality_id: str, indicators_str: str) -> list:
    # (Same function as previously provided - handles fetching)
    data_url = f"{API_BASE_URL}/dades.json?id={municipality_id}&i={indicators_str}"
    try:
        response = requests.get(data_url, timeout=15)
        response.raise_for_status()
        data = response.json()
        results = []
        indicators_data = data.get('fitxes', {}).get('indicadors', {}).get('i', [])
        if not isinstance(indicators_data, list): indicators_data = []
        for indicator in indicators_data:
            if 'error' in indicator: continue
            indicator_id = indicator.get('id')
            name = indicator.get('c')
            values_str = indicator.get('v')
            reference_year = indicator.get('r')
            if values_str is None or indicator_id is None: continue
            values_list = values_str.split(',')
            municipality_value = values_list[0] if len(values_list) > 0 else '_'
            results.append({
                "municipality_id": municipality_id,
                "indicator_id": indicator_id,
                "indicator_name": name,
                "year": reference_year,
                "municipality_value": municipality_value
            })
        return results
    except requests.exceptions.Timeout: return []
    except requests.exceptions.HTTPError as e: return []
    except requests.exceptions.RequestException as e: return []
    except json.JSONDecodeError: return []
    except Exception as e: return []

# --- Main Analysis Logic ---
if __name__ == "__main__":
    print("--- Starting Parallel Missing Value Analysis for Idescat EMEX Indicators ---")

    municipality_ids = get_all_municipality_ids()

    if not municipality_ids:
        print("Could not retrieve municipality IDs. Exiting.")
        exit()

    total_municipalities = len(municipality_ids)
    print(f"\nChecking {len(INDICATORS_TO_CHECK)} indicators across {total_municipalities} municipalities using up to {MAX_WORKERS} parallel workers...")

    all_indicator_data = []
    futures = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        for mun_id in municipality_ids:
            future = executor.submit(fetch_indicators_for_municipality, mun_id, INDICATORS_QUERY_STRING)
            futures.append(future)
        print("Fetching data (progress bar reflects completed municipalities):")
        for future in tqdm(as_completed(futures), total=total_municipalities, unit="municipality"):
            try:
                result = future.result()
                if result:
                    all_indicator_data.extend(result)
            except Exception as exc:
                print(f'\nCaught exception processing a future: {exc}')

    print("\nFinished fetching data. Analyzing missing values...")

    # --- Analysis using Pandas (calculation remains the same) ---
    if not all_indicator_data:
        print("No indicator data was successfully fetched. Cannot analyze.")
        exit()

    df = pd.DataFrame(all_indicator_data)
    df['is_missing'] = df['municipality_value'] == '_'

    # Grouping and aggregation remain the same
    missing_analysis = df.groupby(['indicator_id', 'indicator_name'])['is_missing'].agg(
        total_observations='count', # This is the count of actual fetched values for this indicator
        missing_count='sum'
    ).reset_index()

    # Merging and filling missing indicators remain the same
    all_indicators_df = pd.DataFrame({'indicator_id': INDICATORS_TO_CHECK})
    missing_analysis = pd.merge(all_indicators_df, missing_analysis, on='indicator_id', how='left')
    missing_analysis['total_observations'] = missing_analysis['total_observations'].fillna(0).astype(int)
    missing_analysis['missing_count'] = missing_analysis['missing_count'].fillna(missing_analysis['total_observations']).astype(int)

    missing_analysis['checked_municipalities'] = total_municipalities
    missing_analysis['missing_pct'] = (missing_analysis['missing_count'] / total_municipalities) * 100

    name_map = df[['indicator_id', 'indicator_name']].drop_duplicates().set_index('indicator_id')['indicator_name']
    missing_analysis['indicator_name'] = missing_analysis['indicator_name'].fillna(missing_analysis['indicator_id'].map(name_map))
    missing_analysis['indicator_name'] = missing_analysis['indicator_name'].fillna("N/A (Never Returned)")

    missing_analysis = missing_analysis.sort_values(by='missing_pct', ascending=True)

    # --- Output Section ---
    print("\n--- Missing Value Analysis Results ---")
    print(f"(Based on checking {total_municipalities} municipalities)")
    # Ensure 'total_observations' is in the columns printed
    columns_to_print = ['indicator_id', 'indicator_name', 'total_observations', 'missing_count', 'checked_municipalities', 'missing_pct']
    print(missing_analysis[columns_to_print].round(2).to_string(index=False)) # Use index=False for cleaner print

    # Save results to JSON
    output_file = Path("./idescat_emex_missing_analysis.json")
    # Use 'records' orientation for a list of JSON objects, which is often useful
    # Ensure columns match the printed ones for consistency
    missing_analysis[columns_to_print].to_json(output_file, orient='records', indent=4, force_ascii=False)
    print(f"\nAnalysis results saved to: {output_file}")

    print("\n--- Analysis Complete ---")
    print("Use these results to select indicators with acceptable missing rates for the Landing Zone script.")

In [ ]:
import requests

def obtenir_dades_emex_complet(municipi_id):
    indicadors = ','.join([
        'f171',  # Població total
        'f36',   # Homes
        'f42',   # Dones
        'f182',  # Naixements
        'f183',  # Defuncions
        'f191',  # Migració total
        'f210',  # Taxa d'atur registrat (%)
        'f240',  # Superfície del municipi (km²)
        'f241',  # Densitat de població
        'f248',  # Habitatges principals
        'f271',  # Renda mitjana per habitant (€)
        'f272',  # Mobilitat per motius de treball
        'f328',  # Longitud
        'f329',  # Latitud
        'f3',    # PIB per habitant
        'f186',  # Llars unipersonals
        'f187',  # Llars amb 4 persones o més
        'f260',  # Població de 0 a 14 anys
        'f261',  # Població de 15 a 64 anys
        'f262',  # Població de 65 anys i més
        'f263',  # Índex d'envelliment
        'f264',  # Índex de dependència
        'f265',  # Índex de sobreenvelliment
        'f266',  # Taxa de natalitat
        'f267',  # Taxa de mortalitat
        'f268',  # Taxa de mortalitat infantil
        'f269',  # Taxa de creixement natural
        'f270',  # Taxa migratòria
        'f273',  # Població estrangera
        'f274',  # Població estrangera masculina
        'f275',  # Població estrangera femenina
        'f276',  # Població estrangera (%) sobre el total
        'f277',  # Població estrangera masculina (%) sobre el total
        'f278',  # Població estrangera femenina (%) sobre el total
        'f279',  # Nombre d'empreses
        'f280',  # Nombre d'empreses de 1 a 9 assalariats
        'f281',  # Nombre d'empreses de 10 a 49 assalariats
        'f282',  # Nombre d'empreses de 50 a 199 assalariats
        'f283',  # Nombre d'empreses de 200 o més assalariats
        'f284',  # Taxa d'activitat
        'f285',  # Taxa d'ocupació
        'f286',  # Taxa d'atur registrat masculina
        'f287',  # Taxa d'atur registrat femenina
        'f288',  # Taxa d'atur registrat de menors de 25 anys
        'f289',  # Taxa d'atur registrat de 25 a 44 anys
        'f290',  # Taxa d'atur registrat de 45 anys i més
        'f291',  # Atur registrat de llarga durada
        'f292',  # Atur registrat de curta durada
        'f293',  # Nombre de contractes registrats
        'f294',  # Nombre de contractes indefinits
        'f295',  # Nombre de contractes temporals
        'f296',  # Nombre de contractes a temps complet
        'f297',  # Nombre de contractes a temps parcial
        'f298',  # Nombre de contractes a homes
        'f299',  # Nombre de contractes a dones
        'f300',  # Nombre de contractes a menors de 25 anys
        'f301',  # Nombre de contractes a persones de 25 a 44 anys
        'f302',  # Nombre de contractes a persones de 45 anys i més
        'f303',  # Nombre de centres educatius
        'f304',  # Nombre d'alumnes d'educació infantil
        'f305',  # Nombre d'alumnes d'educació primària
        'f306',  # Nombre d'alumnes d'ESO
        'f307',  # Nombre d'alumnes de batxillerat
        'f308',  # Nombre d'alumnes de cicles formatius de grau mitjà
        'f309',  # Nombre d'alumnes de cicles formatius de grau superior
        'f310',  # Nombre d'alumnes d'ensenyaments artístics
        'f311',  # Nombre d'alumnes d'ensenyaments esportius
        'f312',  # Nombre d'alumnes d'educació especial
        'f313',  # Nombre d'alumnes d'educació d'adults
        'f314',  # Nombre d'alumnes d'ensenyaments de règim especial
        'f315',  # Nombre d'alumnes d'ensenyaments universitaris
        'f316',  # Nombre de biblioteques
        'f317',  # Nombre de museus
        'f318',  # Nombre de sales de cinema
        'f319',  # Nombre de teatres
        'f320',  # Nombre de sales de concerts
        'f321',  # Nombre d'equipaments esportius
        'f322',  # Nombre de places hoteleres
        'f323',  # Nombre de càmpings
        'f324',  # Nombre de places de càmping
        'f325',  # Nombre de turistes
        'f326',  # Nombre de pernoctacions
        'f327',  # Nombre de vehicles turismes
        'f328',  # Nombre de vehicles motocicletes
        'f329'
    ])

    url = f"https://api.idescat.cat/emex/v1/dades.json?id={municipi_id}&i={indicadors}"
    resposta = requests.get(url)
    resposta.raise_for_status()
    dades = resposta.json()

    fitxa = dades.get('fitxes', {})
    resultat = []

    for indicador in fitxa.get('indicadors', {}).get('i', []):
        nom = indicador.get('c')
        valors_raw = indicador.get('v')
        if valors_raw is None:
            continue  # Saltar si no hi ha dades
        valors = valors_raw.split(',')  # municipi, comarca, Catalunya
        any_dada = indicador.get('r')
        resultat.append({
            'indicador': nom,
            'any': any_dada,
            'municipi': valors[0],
            'comarca': valors[1],
            'catalunya': valors[2],
            'actualitzat': indicador.get('updated')
        })
    return resultat

# Exemple d'ús
dades_barcelona = obtenir_dades_emex_complet('431299')

for fila in dades_barcelona:
    print(fila)


In [1]:
# landing_zone_idescat_parallel_with_names.py

import requests
import json
import time
import pandas as pd
from pathlib import Path
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# --- Configuration ---
# Indicators selected based on analysis of idescat_emex_missing_analysis.json
# Keeping indicators with < 1% effective missing rate and relevant to project
CHOSEN_INDICATORS = [
    'f171', # Population
    'f36',  # Men
    'f42',  # Women
    'f187', # Births (Total)
    'f183', # Pop. Spanish Nationality
    'f261', # Surface area (km²)
    'f262', # Density (Pop/km²)
    'f328', # Longitude
    'f329', # Latitude
    'f308', # Total Registered Unemployment
    'f191', # Habitatges familiars
    'f270', # Biblioteques públiques
    'f293', # Pavellons
    'f294', # Pistes poliesportives
    'f301', # Piscines cobertes
]

INDICATORS_QUERY_STRING = ','.join(CHOSEN_INDICATORS)

API_BASE_URL = "https://api.idescat.cat/emex/v1"
MAX_WORKERS = 20 # Using the safer limit

LANDING_ZONE_DIR = Path("./data/landing/idescat_emex")
LANDING_ZONE_DIR.mkdir(parents=True, exist_ok=True)

# --- Helper Functions ---

def get_all_municipality_ids() -> list:
    # (Same function as previously provided)
    print("Fetching list of all municipalities...")
    nodes_url = f"{API_BASE_URL}/nodes.json?tipus=mun"
    try:
        response = requests.get(nodes_url)
        response.raise_for_status()
        data = response.json()
        municipality_ids = []
        def extract_municipalities(node):
            ids = []
            if node.get('scheme') == 'mun':
                ids.append(node.get('id'))
            children = node.get('v', [])
            if isinstance(children, dict): children = [children]
            if isinstance(children, list):
                for child_node in children:
                    ids.extend(extract_municipalities(child_node))
            return ids
        root_nodes = data.get('fitxes', {}).get('v', [])
        if isinstance(root_nodes, dict): root_nodes = [root_nodes]
        for root_node in root_nodes:
             municipality_ids.extend(extract_municipalities(root_node))
        unique_ids = sorted(list(set(municipality_ids)))
        print(f"Found {len(unique_ids)} unique municipality IDs.")
        return unique_ids
    except Exception as e:
        print(f"Error fetching municipality list: {e}")
        return []

# Modified function to extract names
def fetch_chosen_indicators_for_municipality(municipality_id: str, indicators_str: str) -> list:
    """
    Fetches chosen indicators for a single municipality (thread-safe)
    and includes municipality and comarca names.
    """
    data_url = f"{API_BASE_URL}/dades.json?id={municipality_id}&i={indicators_str}"
    try:
        response = requests.get(data_url, timeout=3) # Increased timeout slightly
        response.raise_for_status()
        data = response.json()

        # --- Extract Municipality and Comarca Names from 'cols' ---
        municipality_name = None
        comarca_name = None
        try:
            # Navigate safely through the potential JSON structure
            cols_data = data.get('fitxes', {}).get('cols', {}).get('col', [])
            # Ensure cols_data is always a list, even if API returns single object
            if isinstance(cols_data, dict):
                 cols_data = [cols_data]

            if isinstance(cols_data, list):
                 for col_info in cols_data:
                     # JSON structure might use '@scheme' and '#content' or similar
                     # Adjust based on actual JSON response structure if different
                     scheme = col_info.get('@scheme', col_info.get('scheme')) # Check common variations
                     name_content = col_info.get('#content', col_info.get('content', col_info.get('c'))) # Check common variations

                     if scheme == 'mun':
                         municipality_name = name_content
                     elif scheme == 'com':
                         comarca_name = name_content
        except Exception as e_parse:
             # Log parsing error but continue, names will be None
             # print(f"\nWarning: Error parsing 'cols' for {municipality_id}: {e_parse}")
             pass # Keep names as None
        # -----------------------------------------------------------

        results = []
        indicators_data = data.get('fitxes', {}).get('indicadors', {}).get('i', [])
        if not isinstance(indicators_data, list): indicators_data = []

        for indicator in indicators_data:
            if 'error' in indicator: continue

            indicator_id = indicator.get('id')
            indicator_name_api = indicator.get('c') # Name from indicator section
            values_str = indicator.get('v')
            reference_year = indicator.get('r')
            updated_ts = indicator.get('updated')

            if values_str is None or indicator_id is None: continue

            values_list = values_str.split(',')
            mun_val = values_list[0] if len(values_list) > 0 else None
            com_val = values_list[1] if len(values_list) > 1 else None
            cat_val = values_list[2] if len(values_list) > 2 else None

            results.append({
                "municipality_id": municipality_id,
                "municipality_name": municipality_name, # Add extracted name
                "comarca_name": comarca_name,         # Add extracted name
                "indicator_id": indicator_id,
                "indicator_name": indicator_name_api,
                "reference_year": reference_year,
                "municipality_value": mun_val if mun_val != '_' else None,
                "comarca_value": com_val if com_val != '_' else None,
                "catalunya_value": cat_val if cat_val != '_' else None,
                "source_update_timestamp": updated_ts
            })
        return results

    except requests.exceptions.Timeout: return []
    except requests.exceptions.HTTPError as e: return []
    except requests.exceptions.RequestException as e: return []
    except json.JSONDecodeError: return []
    except Exception as e: return []

# --- Main Landing Zone Logic ---
def run_idescat_landing_zone(save_csv: bool = False):
    """
    Fetches chosen Idescat indicators (incl. names) for all municipalities in parallel
    and saves the data to a Parquet file in the landing zone.
    Optionally saves a CSV file as well.
    """
    print("--- Starting Parallel Landing Zone Task for Idescat EMEX Indicators (with Names) ---")

    municipality_ids = get_all_municipality_ids()

    if not municipality_ids:
        print("Could not retrieve municipality IDs. Exiting.")
        return

    total_municipalities = len(municipality_ids)
    print(f"\nFetching {len(CHOSEN_INDICATORS)} chosen indicators for {total_municipalities} municipalities using up to {MAX_WORKERS} parallel workers...")

    all_data_for_landing = []
    futures = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        for mun_id in municipality_ids:
            future = executor.submit(fetch_chosen_indicators_for_municipality, mun_id, INDICATORS_QUERY_STRING)
            futures.append(future)
        print("Fetching data (progress bar reflects completed municipalities):")
        for future in tqdm(as_completed(futures), total=total_municipalities, unit="municipality"):
            try:
                result = future.result()
                if result:
                    all_data_for_landing.extend(result)
            except Exception as exc:
                print(f'\nCaught exception processing a future: {exc}')

    print("\nFinished fetching data. Preparing and saving...")

    if not all_data_for_landing:
        print("No data was successfully fetched for the chosen indicators. No output files generated.")
        return

    # --- Dataframe Creation and Saving ---
    df_landing = pd.DataFrame(all_data_for_landing)

    # Add name columns to the numeric check if needed, but they should be strings
    numeric_cols = ['municipality_value', 'comarca_value', 'catalunya_value']
    for col in numeric_cols:
        df_landing[col] = pd.to_numeric(df_landing[col], errors='coerce')

    df_landing['reference_year'] = pd.to_numeric(df_landing['reference_year'], errors='coerce').astype('Int64')
    df_landing['source_update_timestamp'] = pd.to_datetime(df_landing['source_update_timestamp'], errors='coerce', utc=True)

    # Reorder columns for better readability (optional)
    desired_order = [
        "municipality_id", "municipality_name", "comarca_name",
        "indicator_id", "indicator_name", "reference_year",
        "municipality_value", "comarca_value", "catalunya_value",
        "source_update_timestamp"
    ]
    # Ensure all desired columns exist before reordering
    existing_columns = [col for col in desired_order if col in df_landing.columns]
    df_landing = df_landing[existing_columns]


    # Define base filename with timestamp
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_filename = f"idescat_emex_indicators_{timestamp_str}"

    # --- Optional CSV Saving ---
    if save_csv:
        csv_output_path = LANDING_ZONE_DIR / f"{base_filename}.csv"
        try:
            df_landing.to_csv(csv_output_path, index=False, encoding='utf-8-sig')
            print(f"\nSuccessfully saved landing data to CSV: {csv_output_path}")
        except Exception as e:
            print(f"\nError saving DataFrame to CSV file {csv_output_path}: {e}")

    # --- Save to Parquet (Primary Format) ---
    parquet_output_path = LANDING_ZONE_DIR / f"{base_filename}.parquet"
    try:
        df_landing.to_parquet(parquet_output_path, index=False, engine='pyarrow')
        print(f"\nSuccessfully saved landing data to PARQUET: {parquet_output_path}")
        print(f"Total rows saved: {len(df_landing)}")
    except Exception as e:
        print(f"\nError saving DataFrame to Parquet file {parquet_output_path}: {e}")

    print("\n--- Landing Zone Task Complete ---")


# --- Main Execution Block ---
if __name__ == "__main__":
    # Set save_csv=True to generate the CSV for visualization this time
    run_idescat_landing_zone(save_csv=True)

--- Starting Parallel Landing Zone Task for Idescat EMEX Indicators (with Names) ---
Fetching list of all municipalities...
Found 947 unique municipality IDs.

Fetching 15 chosen indicators for 947 municipalities using up to 20 parallel workers...
Fetching data (progress bar reflects completed municipalities):


100%|██████████| 947/947 [01:46<00:00,  8.85municipality/s]



Finished fetching data. Preparing and saving...

Successfully saved landing data to CSV: data/landing/idescat_emex/idescat_emex_indicators_20250409_114917.csv

Successfully saved landing data to PARQUET: data/landing/idescat_emex/idescat_emex_indicators_20250409_114917.parquet
Total rows saved: 6536

--- Landing Zone Task Complete ---


In [6]:
# Given this api API_BASE_URL = "https://api.idescat.cat/emex/v1" get the id of 'Barcelona'
import requests

API_BASE_URL = "https://api.idescat.cat/emex/v1"

def get_id_by_name(name: str) -> str:
    """
    Fetches the ID of a municipality by its name from the Idescat API.
    """
    url = f"{API_BASE_URL}/nodes.json?tipus=mun&c={name}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        print(f"Response data: {data}")  # Debugging line
        nodes = data.get('fitxes', {}).get('v', [])
        if isinstance(nodes, dict): nodes = [nodes]
        for node in nodes:
            if node.get('c') == name:
                return node.get('id')
    except Exception as e:
        print(f"Error fetching ID for {name}: {e}")
    return None
# Example usage
municipality_id = get_id_by_name("Barcelona")
if municipality_id:
    print(f"Municipality ID for Barcelona: {municipality_id}")
else:
    print("Municipality ID not found.")

Response data: {'fitxes': {'p': 'tipus=mun', 'v': [{'scheme': 'mun', 'id': '080018', 'content': 'Abrera'}, {'scheme': 'mun', 'id': '080023', 'content': 'Aguilar de Segarra'}, {'scheme': 'mun', 'id': '080039', 'content': 'Alella'}, {'scheme': 'mun', 'id': '080044', 'content': 'Alpens'}, {'scheme': 'mun', 'id': '080057', 'content': "Ametlla del Vallès, l'"}, {'scheme': 'mun', 'id': '080060', 'content': 'Arenys de Mar'}, {'scheme': 'mun', 'id': '080076', 'content': 'Arenys de Munt'}, {'scheme': 'mun', 'id': '080082', 'content': 'Argençola'}, {'scheme': 'mun', 'id': '080095', 'content': 'Argentona'}, {'scheme': 'mun', 'id': '080109', 'content': 'Artés'}, {'scheme': 'mun', 'id': '080116', 'content': 'Avià'}, {'scheme': 'mun', 'id': '080121', 'content': 'Avinyó'}, {'scheme': 'mun', 'id': '080137', 'content': 'Avinyonet del Penedès'}, {'scheme': 'mun', 'id': '080142', 'content': 'Aiguafreda'}, {'scheme': 'mun', 'id': '080155', 'content': 'Badalona'}, {'scheme': 'mun', 'id': '080168', 'content